In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/GraphRAG_fiscal')

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv('/content/drive/MyDrive/GraphRAG_fiscal/cles.env')

api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
!pip install openai
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.2 MB/s eta 0:00:00


In [ ]:
!pip install langchain-text-splitters

In [ ]:
import PyPDF2
import json
import re
from langchain_text_splitters import RecursiveCharacterTextSplitter

def load_cgi_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text


def segment_legal_text(text):
    splitter = RecursiveCharacterTextSplitter(
        separators=["\nArticle"],
        chunk_size=1000,
        chunk_overlap=200
    )
    return splitter.split_text(text)

In [ ]:
txt = load_cgi_pdf('cgi-2025-fr.pdf')
segments = segment_legal_text(txt)
segments[1]

"Article premier. - Définition  \n \n    L'impôt sur les sociétés s’applique sur l'ensemble des produits, \nbénéfices et revenus prévus aux articles 4 et  8 ci-dessous, des sociétés et \nautres personnes morales visées à l’article 2 ci -après."

In [ ]:
len(segments)

455

In [ ]:
import os
import json
import csv
from typing import List, Dict, Any, Optional
from datetime import datetime

class Extracteur:


    def __init__(self, api_key: str = None, model: str = None):


        self.api_key = api_key


        from openai import OpenAI
        self.client = OpenAI(api_key=self.api_key)
        self.model = model



        self.entity_types = [
            "LOI_FISCALE", "IMPOT", "TAUX", "CONTRIBUABLE",
            "REGIME_FISCAL", "OBLIGATION", "DECLARATION", "EXONERATION",
            "DEDUCTION", "ORGANISME", "DATE_LIMITE", "SANCTION",
            "SECTEUR_ACTIVITE", "SEUIL"
        ]

        self.relation_types = [
            "REGIT_PAR", "SOUMIS_A", "APPLIQUE_TAUX", "BENEFICIE_DE",
            "DOIT_DECLARER", "ECHEANCE", "ADMINISTRE_PAR", "SANCTIONNE_PAR",
            "REMPLACE", "CONDITIONNE", "CALCULE_PAR", "CONCERNE"
        ]

    def extraction_prompt(self, text: str) -> str:

        return  f"""Tu es un expert en fiscalité marocaine. Analyse le texte suivant et extrais toutes les entités fiscales pertinentes ainsi que leurs relations, sans te limiter à des catégories pré-définies.

                TEXTE À ANALYSER:
                {text}

                INSTRUCTIONS:
                1. Identifie toutes les entités fiscales dans le texte.
                2. Détermine le type de chaque entité de manière descriptive (ex: impôt, taxe, contribution, etc.).
                3. Identifie toutes les relations entre les entités (ex: "lié à", "applicable sur", "calculé à partir de", etc.).
                4. Extrait les attributs pertinents (montants, pourcentages, dates, délais, etc.).

                Réponds UNIQUEMENT avec un JSON valide suivant ce format exact:
                {{
                "relations": [
                    {{
                    "source_id": "entite_1",
                    "source_text": "texte source",
                    "target_id": "entite_2",
                    "target_text": "texte cible",
                    "type": "TYPE_RELATION",
                    "description": "description"
                    }}
                ]
                }}"""

    def extract(self, text: str) -> Dict[str, Any]:

        response = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "Tu es un expert en droit fiscal marocain."},
                {"role": "user", "content": self.extraction_prompt(text)}
            ],
            temperature=0.1,
            response_format={"type": "json_object"}
        )
        return json.loads(response.choices[0].message.content)




    def save_relations_to_csv(self, relations: List[Dict], filename: str = "relations_fiscales.csv"):


        fieldnames = ["source_id", "source_text", "target_id", "target_text", "type", "description"]

        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

            for relation in relations:
                row = {
                    "source_id": relation.get("source_id", relation.get("source", "")),
                    "source_text": relation.get("source_text", ""),
                    "target_id": relation.get("target_id", relation.get("target", "")),
                    "target_text": relation.get("target_text", ""),
                    "type": relation.get("type", ""),
                    "description": relation.get("description", "")
                }
                writer.writerow(row)



In [ ]:
extracteur = Extracteur(
    api_key=api_key,
    model="gpt-4o-mini"
)

In [ ]:
import time
toutes_relations = []

for i in range(455):
    result = extracteur.extract(segments[i])

    for relation in result.get("relations", []):
        relation["chunk"] = i + 1
        toutes_relations.append(relation)

    time.sleep(2)

extracteur.save_relations_to_csv(toutes_relations, "relations_extraites.csv")


In [ ]:
import pandas as pd
df = pd.read_csv('relations_extraites.csv')

In [ ]:
df.shape

(3829, 6)

In [ ]:
df.head(50)

,source_id,source_text,target_id,target_text,type,description
0,entite_1,Code Général des Impôts,entite_2,loi de finances n° 43-06,institué par,Le Code Général des Impôts est institué par la...
1,entite_2,loi de finances n° 43-06,entite_3,Dahir n° 1-06-232,promulguée par,La loi de finances n° 43-06 est promulguée par...
2,entite_4,impôt sur les sociétés (I.S),entite_5,impôt sur le revenu (I.R),mesures fiscales en matière de,Des mesures fiscales ont été introduites en ma...
3,entite_6,Taxe sur la valeur ajoutée (T.V.A),entite_4,NaN,mesures fiscales en matière de,Des mesures fiscales ont été introduites en ma...
4,entite_7,Droits d'enregistrement (D.E),entite_4,NaN,mesures fiscales en matière de,Des mesures fiscales ont été introduites en ma...
5,entite_8,droits de timbre (D.T.),entite_9,taxe spéciale annuelle sur les véhicules (T.S....,refonte et insertion dans,Les droits de timbre (D.T.) ont été refondus e...
6,entite_10,taxe spéciale annuelle sur les véhicules (T.S....,entite_11,taxe à l’essieu,intégration des dispositions de,Les dispositions de la taxe à l’essieu ont été...
7,entite_12,taxe sur les contrats d'assurances,entite_13,livre III du code général des impôts,intégration des dispositions de,Les dispositions de la taxe sur les contrats d...
8,entite_14,loi-cadre n° 69-19,entite_15,réforme fiscale,portant,La loi-cadre n° 69-19 porte sur la réforme fis...
9,entite_16,taxe aérienne pour la solidarité et la promoti...,entite_17,livre III du code général des impôts,intégration des dispositions de,Les dispositions de la taxe aérienne pour la s...
